In [1]:
import gradio as gr
import numpy as np
import random

### 1. Using different inputs and outputs components

In [ ]:
def greet(name):
    return "Hello, " + name + "!"

# text, image, audio, label, gr.Textbox(lines=2, placeholder="Name here")
# All the widgets you can find here https://www.gradio.app/docs/components

# iface = gr.Interface(fn=greet, inputs="text", outputs="text")
iface = gr.Interface(fn=greet, inputs=gr.Textbox(lines=2, placeholder="Name here"), outputs="text")
iface.launch() # Use iface.close() to close the port

In [ ]:
demo.close() # To close the port

### 2. Using different inputs, outputs and slider

In [ ]:
def greet(name, is_morning, temperature):
    salutation = "Good morning" if is_morning else "Good evening"
    greeting = f"{salutation} {name}. It is {temperature} degrees today"
    celsius = (temperature - 32) * 5/9
    return greeting, round(celsius,2)
# text, image, audic label
demo = gr.Interface(fn=greet, 
                    inputs=["text", "checkbox", gr.Slider(0, 100)],
                    outputs=["text", "number"])
demo.launch()

In [ ]:
demo.close() # gr.close_all()

### 3. Using Image as input

In [ ]:
def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769,0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    print(input_img.shape, sepia_img.shape)
    return sepia_img

demo = gr.Interface(sepia, 
                    gr.Image(height=200,width= 200),
                    "image")
demo.launch()

In [ ]:
demo.close() # gr.close_all()

### 4. Using Audio as input

In [ ]:
def speech_to_text(name):
    # speech to text here...
    return "This is the text"

demo = gr.Interface(fn=speech_to_text,
                    inputs=gr.Audio(label="Audio file"),
                    outputs=gr.Text())
demo.launch(debug=True)

In [ ]:
demo.close() # gr.close_all()

### 5. Using Gradio Blocks (Example 1) IMP

In [ ]:
def greet (name):
    return "Hello " + name + "!"

with gr.Blocks() as demo:
    name = gr.Textbox(label="Name")
    output = gr.Textbox(label="Output Box")
    greet_btn = gr.Button("Greet")
    greet_btn.click(fn=greet,inputs=name, outputs=output)

demo.launch()

In [ ]:
demo.close() # gr.close_all()

### 6. Using Gradio Blocks with Examples IMP
If you already have a code in git repository in github then you can directly import it into huggingface using below commands
<br> git remote add space https://huggingface.co/spaces/FULL_SPACE_NAME
<br> git push --force space main

In [ ]:
from transformers import pipeline
# pipe = pipeline("translation", model="t5-base")
pipe = pipeline("translation_en_to_fr", model="t5-base")

def translate(text):
    return pipe(text)[0]["translation_text"]

# Layout
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            english = gr.Textbox(label="English text")
            translate_btn = gr.Button("Translate")
        with gr.Column():
            french = gr.Textbox(label="French Text")
            # german = gr.Textbox(label="German Text")

    translate_btn.click(translate, inputs=english, outputs=french)
    examples= gr.Examples(examples=["I went to the supermarket yesterday","Helen is a good swimmer."],inputs=english) # inputs=[english]

# demo.launch(share=True, degug=True) # set share=True for sharing Gradio app using public url
demo.launch()

In [ ]:
demo.close()

### 7. Global State vs Session State (we are using in below code)

In [ ]:
def chat(message, history):
    history = history or []
    message = message.lower()
    if message.startswith("how many"):
        response = random.randint(1,10)
    elif message.startswith("how"):
        response = random.choice(["Great", "Good", "Okay", "Bad"])
    elif message.startswith("where"):
        response = random.choice(["Here", "There", "Somewhere"])
    else:
        response = "I don't know"
    history.append((message, response))
    return history, history

chatbot = gr.Chatbot(height=500)

demo = gr.Interface(
    chat,
    ["text", "state"],
    [chatbot, "state"],
    allow_flagging="never",
)

demo.launch()

In [ ]:
demo.close() # gr.close_all()

### Text Summarization with distilbart-cnn

In [ ]:
from transformers import pipeline
import gradio as gr

get_completion = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
# demo = gr.Interface(fn=summarize, inputs="text", outputs="text")
demo = gr.Interface(fn=summarize, inputs=[gr.Textbox(label="Text to Summarize", lines=6)], outputs=[gr.Textbox(label="Result", lines=3)], title="Text summarization with distilbart-cnn", description="Summarize any text using the `shleifer/distilbart-cnn-12-6` model under the hood!")
# demo.launch(share=True, server_port=int(os.environ['PORT1'])) # To open on which ever port you want
# demo.launch(share=True)
demo.launch()

In [ ]:
demo.close()

If you want to use summarization end point below is the snippet (then refer L1_NLP_tasks_with_a_simple_interface.ipynb under E:\ANG\Gradio\Building Generative AI Applications with Gradio)

In [ ]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']

# Helper function
import requests, json

#Summarization endpoint present at the server
def get_completion(inputs, parameters=None,ENDPOINT_URL=os.environ['HF_API_SUMMARY_BASE']): 
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL, headers=headers,
                                data=json.dumps(data)
                               )
    return json.loads(response.content.decode("utf-8"))

### Building a Named Entity Recognition app
In case if you want to run ner end point (Text to Text)
```py
API_URL = os.environ['HF_API_NER_BASE'] #NER endpoint
text = "My name is Andrew, I'm building DeepLearningAI and I live in California"
get_completion(text, parameters=None, ENDPOINT_URL= API_URL)

def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    return {"text": input, "entities": output}
```

In [2]:
from transformers import pipeline
import gradio as gr

get_completion = pipeline("ner", model="dslim/bert-base-NER")

def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": output}

gr.close_all() # Closes all the opened ports
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    #Here we introduce a new tag, examples, easy to use examples for your application
                    examples=["My name is Andrew and I live in California", "My name is Poli and work at HuggingFace"])
# demo.launch(share=True, server_port=int(os.environ['PORT3']))
demo.launch()

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


##### Adding a helper function to merge tokens

In [4]:
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            # If current token continues the entity of the last one, merge them
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # Otherwise, add the token to the list
            merged_tokens.append(token)

    return merged_tokens

def ner(input):
    output = get_completion(input)
    # output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    merged_tokens = merge_tokens(output)
    return {"text": input, "entities": merged_tokens}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    examples=["My name is Andrew, I'm building DeeplearningAI and I live in California", "My name is Poli, I live in Vienna and work at HuggingFace"])

# demo.launch(share=True, server_port=int(os.environ['PORT4']))
demo.launch()

Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### Image Captioning app
Here we'll be using an [Inference Endpoint](https://huggingface.co/inference-endpoints) for `Salesforce/blip-image-captioning-base` a 14M parameter captioning model.
<br> Refer Image captioning end point L2_Image_captioning_app
<br> The code would look very similar if you were running an API (Image to Text end point) instead of it from locally. You can check the [Pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines) documentation page.

In [ ]:
from transformers import pipeline
import gradio as gr
import os
import io
import IPython.display
from PIL import Image
import base64 

get_completion = pipeline("image-to-text",model="Salesforce/blip-image-captioning-base")

def summarize(input):
    output = get_completion(input)
    return output[0]['generated_text']

def image_to_base64_str(pil_image):
    byte_arr = io.BytesIO()
    pil_image.save(byte_arr, format='PNG')
    byte_arr = byte_arr.getvalue()
    return str(base64.b64encode(byte_arr).decode('utf-8'))

def captioner(image):
    base64_image = image_to_base64_str(image)
    result = get_completion(base64_image)
    return result[0]['generated_text']

gr.close_all()
demo = gr.Interface(fn=captioner,
                    inputs=[gr.Image(label="Upload image", type="pil")],
                    outputs=[gr.Textbox(label="Caption")],
                    title="Image Captioning with BLIP",
                    description="Caption any image using the BLIP model",
                    allow_flagging="never",
                    examples=["christmas_dog.jpeg", "bird_flight.jpeg", "cow.jpeg"])

demo.launch()
# demo.launch(share=True, server_port=int(os.environ['PORT1']))

In [ ]:
image_url = "https://free-images.com/sm/9596/dog_animal_greyhound_983023.jpg"
display(IPython.display.Image(url=image_url))
get_completion(image_url)

### Image Generation app
For Text to Image generation end point api refer L3_Image_generation_app
<br> Here we are going to run `runwayml/stable-diffusion-v1-5` using the `diffusers` library.

In [ ]:
import os
import io
import IPython.display
from PIL import Image
import gradio as gr
import base64 

from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")

In [ ]:
def get_completion(prompt):
    return pipeline(prompt).images[0]

prompt = "a dog in a park"

result = get_completion(prompt)

IPython.display.HTML(f'<img src="data:image/png;base64,{result}" />')

In [ ]:
#A helper function to convert the PIL image to base64 so you can send it to the API
def base64_to_pil(img_base64):
    base64_decoded = base64.b64decode(img_base64)
    byte_stream = io.BytesIO(base64_decoded)
    pil_image = Image.open(byte_stream)
    return pil_image

def generate(prompt):
    output = get_completion(prompt)
    result_image = base64_to_pil(output)
    return result_image

gr.close_all()
demo = gr.Interface(fn=generate,
                    inputs=[gr.Textbox(label="Your prompt")],
                    outputs=[gr.Image(label="Result")],
                    title="Image Generation with Stable Diffusion",
                    description="Generate any image with Stable Diffusion",
                    allow_flagging="never",
                    examples=["the spirit of a tamagotchi wandering in the city of Vienna","a mecha robot in a favela"])

# demo.launch(share=True, server_port=int(os.environ['PORT1']))
demo.launch()

In [ ]:
demo.close()

### Building a more Advanced Interface (Blocks are preferred. Go directly to Blocks)

In [ ]:
import gradio as gr 

#A helper function to convert the PIL image to base64 so you can send it to the API
def base64_to_pil(img_base64):
    base64_decoded = base64.b64decode(img_base64)
    byte_stream = io.BytesIO(base64_decoded)
    pil_image = Image.open(byte_stream)
    return pil_image

def generate(prompt, negative_prompt, steps, guidance, width, height):
    params = {
        "negative_prompt": negative_prompt,
        "num_inference_steps": steps,
        "guidance_scale": guidance,
        "width": width,
        "height": height
    }
    
    output = get_completion(prompt)
    pil_image = base64_to_pil(output)
    return pil_image

gr.close_all()
demo = gr.Interface(fn=generate,
                    inputs=[
                        gr.Textbox(label="Your prompt"),
                        gr.Textbox(label="Negative prompt"),
                        gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
                                 info="In how many steps will the denoiser denoise the image?"),
                        gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7, 
                                  info="Controls how much the text prompt influences the result"),
                        gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512),
                        gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512),
                    ],
                    outputs=[gr.Image(label="Result")],
                    title="Image Generation with Stable Diffusion",
                    description="Generate any image with Stable Diffusion",
                    allow_flagging="never"
                    )

demo.launch()
# demo.launch(share=True, server_port=int(os.environ['PORT2']))

In [ ]:
demo.close()

### Blocks `gr.Blocks()`

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Image Generation with Stable Diffusion")
    prompt = gr.Textbox(label="Your prompt")
    with gr.Row():
        with gr.Column():
            negative_prompt = gr.Textbox(label="Negative prompt")
            steps = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
                      info="In many steps will the denoiser denoise the image?")
            guidance = gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7,
                      info="Controls how much the text prompt influences the result")
            width = gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512)
            height = gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512)
            btn = gr.Button("Submit")
        with gr.Column():
            output = gr.Image(label="Result")

    btn.click(fn=generate, inputs=[prompt,negative_prompt,steps,guidance,width,height], outputs=[output])
gr.close_all()

demo.launch()
# demo.launch(share=True, server_port=int(os.environ['PORT3']))

##### More Simplistic UI

In [3]:
with gr.Blocks() as demo:
    gr.Markdown("# Image Generation with Stable Diffusion")
    with gr.Row():
        with gr.Column(scale=4):
            prompt = gr.Textbox(label="Your prompt") #Give prompt some real estate
        with gr.Column(scale=1, min_width=50):
            btn = gr.Button("Submit") #Submit button side by side!
    with gr.Accordion("Advanced options", open=False): #Let's hide the advanced options!
            negative_prompt = gr.Textbox(label="Negative prompt")
            with gr.Row():
                with gr.Column():
                    steps = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
                      info="In many steps will the denoiser denoise the image?")
                    guidance = gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7,
                      info="Controls how much the text prompt influences the result")
                with gr.Column():
                    width = gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512)
                    height = gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512)
    output = gr.Image(label="Result") #Move the output up too
            
    btn.click(fn=generate, inputs=[prompt,negative_prompt,steps,guidance,width,height], outputs=[output])

gr.close_all()
# demo.launch(share=True, server_port=int(os.environ['PORT4']))
demo.launch()

NameError: name 'generate' is not defined

In [18]:
gr.close_all()

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860


<br> E\ANG\Gradio\Building Generative AI Applications with Gradio\4 Describe and Generate Game.mp4
<br> E:\ANG\Gradio\Building Generative AI Applications with Gradio\5 Chat with any LLM.mp4